In [5]:
import config

In [6]:
from mysql.connector import errorcode

In [29]:
import jake_playground

In [7]:
import mysql.connector as mysql

mydb = mysql.connect(
  host=config.host,
  user=config.admin,
  passwd=config.password,
  database='MLB_Stats'
)

mycursor = mydb.cursor()

#mycursor.execute("CREATE DATABASE MLB_Stats")

In [31]:
tables = {}
tables['games'] = (
    "CREATE TABLE games ("
    "  gameid int(11) NOT NULL,"
    "  game_end_datetime datetime(6) NOT NULL,"
    "  game_date datetime(3) NOT NULL,"
    "  home_id int(3) NOT NULL,"
    "  away_id int(3) NOT NULL,"
    "  status varchar(20) NOT NULL,"
    "  PRIMARY KEY (gameid)"
    ") ENGINE=InnoDB")

In [32]:
tables['game_info'] = (
    "CREATE TABLE game_info ("
    "  gameid int(11) NOT NULL,"
    "  info_id int NOT NULL AUTO_INCREMENT,"
    "  home_team_runs int(3) NOT NULL,"
    "  away_team_runs int(3) NOT NULL,"
    "  venue_id varchar(10) NOT NULL,"
    "  weather_category varchar(30) NOT NULL,"
    "  temp int(3) NOT NULL,"
    "  wind_mph int(3) NOT NULL,"
    "  wind_direction varchar(40) NOT NULL,"
    "  PRIMARY KEY (info_id),"
    "  FOREIGN KEY (gameid) REFERENCES games(gameid)"
    ") ENGINE=InnoDB")

In [33]:
tables['teams'] = (
    " CREATE TABLE teams("
    " team_id int(11) NOT NULL,"
    " team_name VARCHAR(30) NOT NULL,"
    " league VARCHAR(2) NOT NULL,"
    " PRIMARY KEY (team_id)"
    ") ENGINE=InnoDB")

In [34]:
tables['venues'] = (
    " CREATE TABLE venue("
    " venue_id int(11) NOT NULL AUTO_INCREMENT,"
    " team_id int(11) NOT NULL,"
    " name VARCHAR(30) NOT NULL,"
    " PRIMARY KEY (venue_id),"
    " FOREIGN KEY (team_id) REFERENCES teams(team_id)"
    ") ENGINE=InnoDB")

In [35]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        mycursor.execute(table_description)
    except mysql.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table games: OK
Creating table game_info: OK
Creating table teams: already exists.
Creating table venues: OK


In [36]:
df = jake_playground.df

In [37]:
df = df[['gameid','game_end_datetime','game_date','status','home_id','away_id']]
df.head()

,gameid,game_end_datetime,game_date,status,home_id,away_id
0,244193,2009-04-06 00:00:00,2009-04-05,Final,143,144
1,244201,2009-04-06 17:10:00,2009-04-06,Final,113,121
2,244195,2009-04-06 18:05:00,2009-04-06,Final,140,114
5,244196,2009-04-06 19:40:00,2009-04-06,Final,109,115
6,244200,2009-04-06 20:05:00,2009-04-06,Final,110,147


In [38]:
tuples = [tuple(x) for x in df.values]

In [39]:
len(tuples), tuples[0]

(26325, (244193, '2009-04-06 00:00:00', '2009-04-05', 'Final', 143, 144))

In [40]:
statement = """INSERT IGNORE INTO games 
               (gameid, game_end_datetime, game_date, status, home_id, away_id)
               VALUES (%s, %s, %s, %s, %s, %s)"""

In [41]:
for i in range(len(tuples)):
    try:
        mycursor.execute(statement, tuples[i])
        mydb.commit()
    except:
        print(i)

In [8]:
stmt = """SELECT gameid from games"""
mycursor.execute(stmt)

In [9]:
ids = []
for obj in mycursor:
    ids.append(obj[0])

In [10]:
len(ids)

26325

In [30]:
import game_log_parse

In [ ]:
statement = """INSERT IGNORE INTO game_info 
               (gameid, home_team_runs, away_team_runs, venue_id, weather_category, temp, wind_mph, wind_direction)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s,)"""

In [12]:
a= game_log_parse.box_upload_get(ids[0])

In [13]:
a

{'away_team_runs': 4,
 'gameid': 244193,
 'home_team_runs': 1,
 'temp': 60,
 'venue_id': 2681,
 'weather_category': 'clear',
 'wind_direction': 'out to cf',
 'wind_mph': 10}

In [16]:
def upload_game(game_dict):
    stmt = statement = """INSERT IGNORE INTO game_info 
               (gameid, home_team_runs, away_team_runs, venue_id, weather_category, temp, wind_mph, wind_direction)
               VALUES (%(gameid)s, %(home_team_runs)s, %(away_team_runs)s, %(venue_id)s, %(weather_category)s, %(temp)s, %(wind_mph)s, %(wind_direction)s)"""
    mycursor.execute(stmt, game_dict)

In [17]:
upload_game(a)
mydb.commit()

In [18]:
games_uploaded = []
not_uploaded = []
for game in ids[1:]:
    try:
        upload = game_log_parse.box_upload_get(game)
        upload_game(upload)
        mydb.commit()
        games_uploaded.append(game)
    except:
        not_uploaded.append(game)


In [19]:
len(not_uploaded)

4

In [32]:
games_uploaded[0]

244194